In [1]:
import pandas as pd 

rock_samples = pd.read_csv('data/rocksamples.csv') 

In [2]:
rock_samples.head()

,ID,Mission,Type,Subtype,Weight (g),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,125.8,88.36
1,10002,Apollo11,Soil,Unsieved,5629.0,93.73
2,10003,Apollo11,Basalt,Ilmenite,213.0,65.56
3,10004,Apollo11,Core,Unsieved,44.8,71.76
4,10005,Apollo11,Core,Unsieved,53.4,40.31


In [3]:
rock_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            2229 non-null   int64  
 1   Mission       2229 non-null   object 
 2   Type          2229 non-null   object 
 3   Subtype       2226 non-null   object 
 4   Weight (g)    2229 non-null   float64
 5   Pristine (%)  2229 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 104.6+ KB


From this output, we can see that 2,229 samples were collected from the Apollo missions. Looking at a sample of the data, we can see that each row contains:

- ID - The unique ID used to keep track of the sample at NASA.
- Mission - The mission responsible for retrieving the sample.
- Type - The type of sample (type of rock or other classification).
- Subtype - A more specific type classification.
- Weight (g) - The original weight of the sample, in grams.
- Pristine (%) - The percentage of the sample that remains (some sample is used up during research).

In [4]:
rock_samples.describe()

,ID,Weight (g),Pristine (%)
count,2229.000000,2229.000000,2229.000000
mean,52058.432032,168.253024,84.512764
std,26207.651471,637.286458,22.057299
min,10001.000000,0.000000,0.000000
25%,15437.000000,3.000000,80.010000
50%,65527.000000,10.200000,92.300000
75%,72142.000000,93.490000,98.140000
max,79537.000000,11729.000000,180.000000


## Convert the sample weight



While details of rocket design are proprietary, some information is publicly available, such as the weight of the modules (parts of the rocket) that will carry the samples back to Earth, and the total amount of weight that the rocket can lift above the atmosphere.

We will get into the specifics of that in a later unit, but the critical part for the purposes of the samples is understanding that rocket weight is often measured in kilograms, not grams. We should then manipulate the original data by converting the sample weights into kilograms for easier data analysis later.

In [5]:
rock_samples["Weight (g)"] = rock_samples["Weight (g)"].apply(lambda x : x*0.001)
rock_samples.rename(columns = {"Weight (g)" : "Weight (kg)"}, inplace = True)
rock_samples.head()

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56
3,10004,Apollo11,Core,Unsieved,0.0448,71.76
4,10005,Apollo11,Core,Unsieved,0.0534,40.31


Create a new DataFrame called missions that will be a summary of data for each of the six Apollo missions that brought samples back. Create a column in this DataFrame called Mission that has one row for each mission.

In [6]:
missions = pd.DataFrame()
missions["Mission"] = rock_samples["Mission"].unique()
missions.head()

,Mission
0,Apollo11
1,Apollo12
2,Apollo14
3,Apollo15
4,Apollo16


In [7]:
missions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Mission  6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes


## Sum total sample weight by mission

Now you can add a new column to the missions DataFrame to represent the sum of all samples collected on that mission.

In [8]:
sample_total_weight = rock_samples.groupby('Mission')['Weight (kg)'].sum()
missions = pd.merge(missions, sample_total_weight, on="Mission")
missions.rename(columns={'Weight (kg)':'Sample weight (kg)'}, inplace=True)
missions

,Mission,Sample weight (kg)
0,Apollo11,21.55424
1,Apollo12,34.34238
2,Apollo14,41.83363
3,Apollo15,75.39910
4,Apollo16,92.46262
5,Apollo17,109.44402


In [9]:
missions["Sample weight (kg)"].sum()

375.03598999999997

## Get the difference in weights across missions

We're not rocket experts, so it's important to take a look at a lot of different cross sections of data that are available to you. In this case, we can see that the total weight of the samples increased with each mission, but it's hard to immediately see by how much. We can add one more column to the missions DataFrame that simply grabs the difference between the current row and the row preceding it:

In [10]:
missions["Weight Differences"] = missions["Sample weight (kg)"].diff()
missions

,Mission,Sample weight (kg),Weight Differences
0,Apollo11,21.55424,NaN
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


Notice that in the first row, for Apollo11, the value in the Weight diff column is NaN. This is called a null value. Because Apollo11 was the first mission, there is no difference between the weight of the rock collected on Apollo11 and that of the previous mission. We can fill this NaN value with 0:

In [11]:
missions["Weight Differences"] = missions["Weight Differences"].fillna(value=0)
missions.head()

,Mission,Sample weight (kg),Weight Differences
0,Apollo11,21.55424,0.00000
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352


## Revisiting the Apollo program

The Apollo program focused on using the Saturn V rocket to send humans into space and onto the Moon. The Saturn V rocket that was used in the Apollo program is known as a three-stage rocket. This means that the rocket has three parts, each of which burns at a different times to achieve a different goal.

The first stage is the main thrust portion that gets the rocket to about 68 kilometers into the sky, and then it falls away back to Earth, making the rocket significantly lighter. The second stage starts burning its engines until the rocket nearly reaches Earth's orbit and likewise falls back down to Earth. The final stage gets the spacecraft into Earth's orbit and thrusts it toward the Moon.

The last bit of relevant information about the Apollo program and Saturn V is that for lunar landings there are two important modules:

- Command module: The module that astronauts live in. When two astronauts are down on the surface of the Moon, the third astronaut stays in the command module. This module is returned to Earth.

- Lunar module: The module that detaches from the command module after it has reached orbit around the Moon. This module lands on the surface of the Moon and can carry two astronauts. When the lunar module returns from the surface to the command module, it leaves part of the base (the landing gear) on the surface of the Moon.
The modules are critical parts of the ship because they are designed precisely to ensure that the astronauts can enter the Moon's orbit, orbit the Moon, land on the Moon, launch from the Moon, and return safely to Earth. The amount of space and weight on each of these modules is precise to ensure the safety and success of the mission. It is fair to conclude that the specifications around these modules affect the amount of mineral samples that can be collected, because the samples have to be carried on each of these modules before returning to Earth.

## Add in command and lunar module data

By using the [NASA Space Science Data Coordinated Archive](https://nssdc.gsfc.nasa.gov/nmc/SpacecraftQuery.jsp), we gathered information about each module used in each mission. As you did when you created the samples tables, create six new columns, three for the lunar modules and three for the command modules:

Module name
Module mass
Module mass diff
Fill in any NaN values with 0:

In [12]:
missions['Lunar module (LM)'] = ['Eagle (LM-5)', 'Intrepid (LM-6)', 'Antares (LM-8)', 'Falcon (LM-10)', 'Orion (LM-11)', 'Challenger (LM-12)']
missions['LM mass (kg)'] = [15103, 15235, 15264, 16430, 16445, 16456]
missions['LM mass diff'] = missions['LM mass (kg)'].diff()
missions['LM mass diff'] = missions['LM mass diff'].fillna(value=0)

missions['Command module (CM)'] = ['Columbia (CSM-107)', 'Yankee Clipper (CM-108)', 'Kitty Hawk (CM-110)', 'Endeavor (CM-112)', 'Casper (CM-113)', 'America (CM-114)']
missions['CM mass (kg)'] = [5560, 5609, 5758, 5875, 5840, 5960]
missions['CM mass diff'] = missions['CM mass (kg)'].diff()
missions['CM mass diff'] = missions['CM mass diff'].fillna(value=0)

missions

,Mission,Sample weight (kg),Weight Differences,Lunar module (LM),LM mass (kg),LM mass diff,Command module (CM),CM mass (kg),CM mass diff
0,Apollo11,21.55424,0.00000,Eagle (LM-5),15103,0.0,Columbia (CSM-107),5560,0.0
1,Apollo12,34.34238,12.78814,Intrepid (LM-6),15235,132.0,Yankee Clipper (CM-108),5609,49.0
2,Apollo14,41.83363,7.49125,Antares (LM-8),15264,29.0,Kitty Hawk (CM-110),5758,149.0
3,Apollo15,75.39910,33.56547,Falcon (LM-10),16430,1166.0,Endeavor (CM-112),5875,117.0
4,Apollo16,92.46262,17.06352,Orion (LM-11),16445,15.0,Casper (CM-113),5840,-35.0
5,Apollo17,109.44402,16.98140,Challenger (LM-12),16456,11.0,America (CM-114),5960,120.0


We can add some totals for each mission across both the lunar and command modules:

In [13]:
missions['Total weight (kg)'] = missions["LM mass (kg)"] + missions["CM mass (kg)"]
missions['Total weight diff'] = missions["LM mass diff"] + missions["CM mass diff"]
missions

,Mission,Sample weight (kg),Weight Differences,Lunar module (LM),LM mass (kg),LM mass diff,Command module (CM),CM mass (kg),CM mass diff,Total weight (kg),Total weight diff
0,Apollo11,21.55424,0.00000,Eagle (LM-5),15103,0.0,Columbia (CSM-107),5560,0.0,20663,0.0
1,Apollo12,34.34238,12.78814,Intrepid (LM-6),15235,132.0,Yankee Clipper (CM-108),5609,49.0,20844,181.0
2,Apollo14,41.83363,7.49125,Antares (LM-8),15264,29.0,Kitty Hawk (CM-110),5758,149.0,21022,178.0
3,Apollo15,75.39910,33.56547,Falcon (LM-10),16430,1166.0,Endeavor (CM-112),5875,117.0,22305,1283.0
4,Apollo16,92.46262,17.06352,Orion (LM-11),16445,15.0,Casper (CM-113),5840,-35.0,22285,-20.0
5,Apollo17,109.44402,16.98140,Challenger (LM-12),16456,11.0,America (CM-114),5960,120.0,22416,131.0


## Compare the data

The interesting thing about predicting how much sample each Artemis mission can bring back is that we don't yet know the full specs of the spacecraft that the Artemis plans on using. Using some information from the [NASA Factsheet on the Space Launch System (SLS) and Orion Modules](https://www.nasa.gov/sites/default/files/atoms/files/0080_sls_fact_sheet_sept2020_09082020_final_0.pdf), we have data on weights and payloads.

A payload is basically the total amount of weight that a rocket can get up through our atmosphere and into space. So the likelihood that the payload number is more accurate than the exact weights of each module is high, because deciding the payload will likely affect each of the other design decisions.

We know that the Saturn V payload was 43,500 kg, and the weights of the modules varied from mission to mission. So, to determine the ratios that will allow us to make predictions about the Artemis missions, we can use:

Saturn V payload
Mission sample weight
Mission module weight

In [14]:
# Sample-to-weight ratio
saturnVPayload = 43500
missions['Crewed area : Payload'] = missions['Total weight (kg)'] / saturnVPayload
missions['Sample : Crewed area'] = missions['Sample weight (kg)'] / missions['Total weight (kg)']
missions['Sample : Payload'] = missions['Sample weight (kg)'] / saturnVPayload
missions

,Mission,Sample weight (kg),Weight Differences,Lunar module (LM),LM mass (kg),LM mass diff,Command module (CM),CM mass (kg),CM mass diff,Total weight (kg),Total weight diff,Crewed area : Payload,Sample : Crewed area,Sample : Payload
0,Apollo11,21.55424,0.00000,Eagle (LM-5),15103,0.0,Columbia (CSM-107),5560,0.0,20663,0.0,0.475011,0.001043,0.000495
1,Apollo12,34.34238,12.78814,Intrepid (LM-6),15235,132.0,Yankee Clipper (CM-108),5609,49.0,20844,181.0,0.479172,0.001648,0.000789
2,Apollo14,41.83363,7.49125,Antares (LM-8),15264,29.0,Kitty Hawk (CM-110),5758,149.0,21022,178.0,0.483264,0.001990,0.000962
3,Apollo15,75.39910,33.56547,Falcon (LM-10),16430,1166.0,Endeavor (CM-112),5875,117.0,22305,1283.0,0.512759,0.003380,0.001733
4,Apollo16,92.46262,17.06352,Orion (LM-11),16445,15.0,Casper (CM-113),5840,-35.0,22285,-20.0,0.512299,0.004149,0.002126
5,Apollo17,109.44402,16.98140,Challenger (LM-12),16456,11.0,America (CM-114),5960,120.0,22416,131.0,0.515310,0.004882,0.002516


In [15]:
missions.describe()

,Sample weight (kg),Weight Differences,LM mass (kg),LM mass diff,CM mass (kg),CM mass diff,Total weight (kg),Total weight diff,Crewed area : Payload,Sample : Crewed area,Sample : Payload
count,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
mean,62.505998,14.648297,15822.166667,225.500000,5767.000000,66.666667,21589.166667,292.166667,0.496303,0.002849,0.001437
std,35.112368,11.299489,683.028965,463.263532,156.292034,73.823212,826.437031,493.138284,0.018999,0.001520,0.000807
min,21.554240,0.000000,15103.000000,0.000000,5560.000000,-35.000000,20663.000000,-20.000000,0.475011,0.001043,0.000495
25%,36.215193,8.815473,15242.250000,12.000000,5646.250000,12.250000,20888.500000,32.750000,0.480195,0.001733,0.000833
50%,58.616365,14.884770,15847.000000,22.000000,5799.000000,83.000000,21653.500000,154.500000,0.497782,0.002685,0.001348
75%,88.196740,17.042990,16441.250000,106.250000,5866.250000,119.250000,22300.000000,180.250000,0.512644,0.003957,0.002028
max,109.444020,33.565470,16456.000000,1166.000000,5960.000000,149.000000,22416.000000,1283.000000,0.515310,0.004882,0.002516


Note: We're calling the two modules crewed area in the DataFrame because those are the parts of the spacecraft where the crew can be, and likely where the samples would also reside.

## Save the ratios

We can then use the mean() function to take the average of all those ratios across all the missions.

In [16]:
crewedArea_payload_ratio = missions['Crewed area : Payload'].mean()
sample_crewedArea_ratio = missions['Sample : Crewed area'].mean()
sample_payload_ratio = missions['Sample : Payload'].mean()
print("Crewed Area Payload Ratio:", crewedArea_payload_ratio)
print("Sample Crewed Area Ratio:", sample_crewedArea_ratio)
print("Sample Payload Ratio:", sample_payload_ratio)


Crewed Area Payload Ratio: 0.4963026819923371
Sample Crewed Area Ratio: 0.002848764392685611
Sample Payload Ratio: 0.0014369195019157087


## Predict Artemis sample capacity

We can use the [NASA Factsheet on the Space Launch System (SLS) and Orion Modules](https://www.nasa.gov/sites/default/files/atoms/files/0080_sls_fact_sheet_sept2020_09082020_final_0.pdf) to gather estimated data on the rockets and modules that will be used in the Artemis program.

As a reminder, the [Artemis program](https://www.nasa.gov/specials/artemis/) is NASA's second set of missions to land humans on the surface of the Moon. The program will launch in 2024 and will send not only the next pair of humans, but also the first woman to set foot on the Moon. The preparation for this mission is even bigger than focusing on a Moon landing. It will also provide space for a commercial payload on the ship and is the first step along the [Moon to Mars program](https://www.nasa.gov/topics/moon-to-mars). So, while the Artemis missions will likely bring home additional samples, there are other goals that might affect the amount of capacity that's required to do so.

## Create an Artemis mission DataFrame

We don't have all the details about the Artemis mission, but we do know currently that three iterations of the rocket will be cycled through for each mission. Each rocket will have one version meant to sustain a crew and one meant only for cargo. For the purposes of this module, we will focus only on the three rockets meant to house crew, to be more aligned with the Apollo missions. We also know that the expected payload of the Space Launch System (SLS) is expected to grow with each iteration, but that the current weight of Orion (the command and lunar modules combined) has one estimated weight today.

Again, we will call the command and lunar modules the crewed area, and we can create a DataFrame with the information we have about the three crewed missions:

In [17]:
artemis_crewedArea = 26520
artemis_mission = pd.DataFrame({'Mission':['artemis1','artemis1b','artemis2'],
                                 'Total weight (kg)':[artemis_crewedArea,artemis_crewedArea,artemis_crewedArea],
                                 'Payload (kg)':[26988, 37965, 42955]})
artemis_mission

,Mission,Total weight (kg),Payload (kg)
0,artemis1,26520,26988
1,artemis1b,26520,37965
2,artemis2,26520,42955


And we can estimate the weight of samples based on the ratios we determined from the Artemis missions:

In [18]:
artemis_mission['Sample weight from total (kg)'] = artemis_mission['Total weight (kg)'] * sample_crewedArea_ratio
artemis_mission['Sample weight from payload (kg)'] = artemis_mission['Payload (kg)'] * sample_payload_ratio
artemis_mission


,Mission,Total weight (kg),Payload (kg),Sample weight from total (kg),Sample weight from payload (kg)
0,artemis1,26520,26988,75.549232,38.779584
1,artemis1b,26520,37965,75.549232,54.552649
2,artemis2,26520,42955,75.549232,61.722877


Finally, we can get the average of the two predictions:

In [19]:
artemis_mission['Estimated sample weight (kg)'] = (artemis_mission['Sample weight from total (kg)'] + artemis_mission['Sample weight from payload (kg)'])/2
artemis_mission

,Mission,Total weight (kg),Payload (kg),Sample weight from total (kg),Sample weight from payload (kg),Estimated sample weight (kg)
0,artemis1,26520,26988,75.549232,38.779584,57.164408
1,artemis1b,26520,37965,75.549232,54.552649,65.050940
2,artemis2,26520,42955,75.549232,61.722877,68.636054


## Prioritize Moon rock sample gathering based on data

Determining which types of samples to collect from the Moon requires expertise, but we can start to make some assumptions to learn how to clean and manipulate data.

First, we can determine how much remains of each sample that was returned from the Apollo missions, given the amount that was originally collected and the percentage of remaining pristine sample.

In [20]:
rock_samples["Remaining (kg)"] = rock_samples["Weight (kg)"] * (rock_samples["Pristine (%)"] *0.01)
rock_samples

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%),Remaining (kg)
0,10001,Apollo11,Soil,Unsieved,0.12580,88.36,0.111157
1,10002,Apollo11,Soil,Unsieved,5.62900,93.73,5.276062
2,10003,Apollo11,Basalt,Ilmenite,0.21300,65.56,0.139643
3,10004,Apollo11,Core,Unsieved,0.04480,71.76,0.032148
4,10005,Apollo11,Core,Unsieved,0.05340,40.31,0.021526
...,...,...,...,...,...,...,...
2224,79528,Apollo17,Breccia,Regolith,0.00238,100.00,0.002380
2225,79529,Apollo17,Breccia,Regolith,0.00184,100.00,0.001840
2226,79535,Apollo17,Breccia,Regolith,0.00169,100.00,0.001690
2227,79536,Apollo17,Breccia,Regolith,0.00166,100.00,0.001660


In [21]:
rock_samples.describe()

,ID,Weight (kg),Pristine (%),Remaining (kg)
count,2229.000000,2229.000000,2229.000000,2229.000000
mean,52058.432032,0.168253,84.512764,0.138103
std,26207.651471,0.637286,22.057299,0.525954
min,10001.000000,0.000000,0.000000,0.000000
25%,15437.000000,0.003000,80.010000,0.002432
50%,65527.000000,0.010200,92.300000,0.008530
75%,72142.000000,0.093490,98.140000,0.078240
max,79537.000000,11.729000,180.000000,11.169527


This helps us see that, on average, each sample weighs about .16 kg and has about 84% of the original amount remaining. We can use this knowledge to extract only the samples that are likely running low, which means that they have been used a lot by researchers.

In [22]:
low_samples = rock_samples.loc[(rock_samples["Weight (kg)"] >= 0.16) & (rock_samples["Pristine (%)"] <= 50)]
low_samples

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%),Remaining (kg)
11,10017,Apollo11,Basalt,Ilmenite,0.9730,43.71,0.425298
14,10020,Apollo11,Basalt,Ilmenite,0.4250,27.88,0.118490
15,10021,Apollo11,Breccia,Regolith,0.2500,30.21,0.075525
29,10045,Apollo11,Basalt,Olivine,0.1850,12.13,0.022441
37,10057,Apollo11,Basalt,Ilmenite,0.9190,35.15,0.323028
39,10059,Apollo11,Breccia,Regolith,0.1880,36.94,0.069447
52,10072,Apollo11,Basalt,Ilmenite,0.4470,15.22,0.068033
59,10086,Apollo11,Soil,Unsieved,0.8230,0.01,0.000082
68,12002,Apollo12,Basalt,Olivine,1.5300,49.04,0.750312
69,12003,Apollo12,Soil,Unsieved,0.3000,28.52,0.085560


In [23]:
low_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              27 non-null     int64  
 1   Mission         27 non-null     object 
 2   Type            27 non-null     object 
 3   Subtype         27 non-null     object 
 4   Weight (kg)     27 non-null     float64
 5   Pristine (%)    27 non-null     float64
 6   Remaining (kg)  27 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.7+ KB


Twenty-seven samples seem like a small amount to base a recommendation on. We can probably find some other samples that are needed for more research here on Earth. To discover them, we can use the unique() function to see how many unique types we have across the low_samples and rock_samples DataFrames.

In [24]:
low_samples["Type"].unique()

array(['Basalt', 'Breccia', 'Soil', 'Core'], dtype=object)

In [25]:
rock_samples.Type.unique()

array(['Soil', 'Basalt', 'Core', 'Breccia', 'Special', 'Crustal'],
      dtype=object)

We can see that, although six unique types were collected across all samples, the samples that are running low are from only four unique types. But this doesn't tell us everything about the samples we might want to focus on. For example, in our low_samples DataFrame, how many of each type are considered low?

In [26]:
low_samples.groupby('Type')['Weight (kg)'].count()

Type
Basalt     14
Breccia     8
Core        1
Soil        4
Name: Weight (kg), dtype: int64

Notice that there are more Basalt and Breccia type rocks with low samples than those of Core and Soil. Additionally, because the likelihood is high that every mission has some Core and Soil collection requirements, we can focus on the Basalt and Breccia rock types for the samples that we need to have collected:

In [27]:
needed_samples = low_samples[low_samples.Type.isin(['Basalt', 'Breccia'])]
needed_samples

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%),Remaining (kg)
11,10017,Apollo11,Basalt,Ilmenite,0.9730,43.71,0.425298
14,10020,Apollo11,Basalt,Ilmenite,0.4250,27.88,0.118490
15,10021,Apollo11,Breccia,Regolith,0.2500,30.21,0.075525
29,10045,Apollo11,Basalt,Olivine,0.1850,12.13,0.022441
37,10057,Apollo11,Basalt,Ilmenite,0.9190,35.15,0.323028
39,10059,Apollo11,Breccia,Regolith,0.1880,36.94,0.069447
52,10072,Apollo11,Basalt,Ilmenite,0.4470,15.22,0.068033
68,12002,Apollo12,Basalt,Olivine,1.5300,49.04,0.750312
72,12006,Apollo12,Basalt,Olivine,0.2064,0.53,0.001094
113,12047,Apollo12,Basalt,Ilmenite,0.1930,33.70,0.065041


In [28]:
needed_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              22 non-null     int64  
 1   Mission         22 non-null     object 
 2   Type            22 non-null     object 
 3   Subtype         22 non-null     object 
 4   Weight (kg)     22 non-null     float64
 5   Pristine (%)    22 non-null     float64
 6   Remaining (kg)  22 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.4+ KB


## Develop a recommendation of Moon rock samples to be collected

Let's take a step back and see how the number of samples compares to the amount of sample. We can compare the total weight from the needed_samples DataFrame to the rock_samples DataFrame. That is, we'll compare the samples we've identified as running low to all the samples collected on Apollo missions.

In [29]:
needed_samples.groupby('Type')['Weight (kg)'].sum()

Type
Basalt     17.4234
Breccia    10.1185
Name: Weight (kg), dtype: float64

In [30]:
rock_samples.groupby('Type')['Weight (kg)'].sum()

Type
Basalt      93.14077
Breccia    168.88075
Core        19.93587
Crustal      4.74469
Soil        87.58981
Special      0.74410
Name: Weight (kg), dtype: float64

One bit of information really stands out: we've never had a lot of Crustal rocks in the first place.

We can add Crustal rocks to the set of needed samples:

In [31]:
needed_samples = needed_samples.append(rock_samples[rock_samples['Type'] == 'Crustal' ])
needed_samples

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%),Remaining (kg)
11,10017,Apollo11,Basalt,Ilmenite,0.97300,43.71,0.425298
14,10020,Apollo11,Basalt,Ilmenite,0.42500,27.88,0.118490
15,10021,Apollo11,Breccia,Regolith,0.25000,30.21,0.075525
29,10045,Apollo11,Basalt,Olivine,0.18500,12.13,0.022441
37,10057,Apollo11,Basalt,Ilmenite,0.91900,35.15,0.323028
...,...,...,...,...,...,...,...
2089,78238,Apollo17,Crustal,Norite,0.00000,86.03,0.000000
2092,78256,Apollo17,Crustal,Pristine,0.00000,0.00,0.000000
2126,78517,Apollo17,Crustal,Cataclasite,0.00182,82.97,0.001510
2130,78527,Apollo17,Crustal,Cataclasite,0.00516,82.05,0.004234


In [32]:
needed_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 11 to 2189
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              68 non-null     int64  
 1   Mission         68 non-null     object 
 2   Type            68 non-null     object 
 3   Subtype         68 non-null     object 
 4   Weight (kg)     68 non-null     float64
 5   Pristine (%)    68 non-null     float64
 6   Remaining (kg)  68 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 4.2+ KB


## Summary of needed samples

The final step is to consolidate everything we know into one table that can be shared with the astronauts. First, we need a column for each type of rock that we have already identified as rocks we want more samples of:

In [33]:
needed_samples_overview = pd.DataFrame()
needed_samples_overview['Type'] = needed_samples.Type.unique()
needed_samples_overview

,Type
0,Basalt
1,Breccia
2,Crustal


Next, we want the total weight of each type of rock that was originally collected:

In [34]:
total_weight = needed_samples.groupby('Type')['Weight (kg)'].sum().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, total_weight, on='Type')
needed_samples_overview.rename(columns={'Weight (kg)':'Total weight (kg)'}, inplace=True)
needed_samples_overview

,Type,Total weight (kg)
0,Basalt,17.42340
1,Breccia,10.11850
2,Crustal,4.74469


When astronauts are up on the Moon, one way they can identify rocks is by their size. If we can give them an estimated size of each type of rock, that might make their collection process easier.

In [35]:
avg_weight = needed_samples.groupby('Type')['Weight (kg)'].mean().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, avg_weight, on='Type')
needed_samples_overview.rename(columns={'Weight (kg)':'Average weight (kg)'}, inplace=True)
needed_samples_overview

,Type,Total weight (kg),Average weight (kg)
0,Basalt,17.42340,1.244529
1,Breccia,10.11850,1.264813
2,Crustal,4.74469,0.103145


Crustals are small! They're probably a lot harder to spot, so no wonder we don't have a lot of them.

We probably want to give the astronauts some indication of how many of each type we want them to collect. So, for the three types we're looking for, we should grab the total number we have of each type and get the remaining percentage of each type of rock.

In [37]:
total_rock_count = rock_samples.groupby('Type')['ID'].count().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, total_rock_count, on = 'Type')
needed_samples_overview.rename(columns={'ID':'Number of Samples'}, inplace=True)
needed_samples_overview

,Type,Total weight (kg),Average weight (kg),Number of Samples
0,Basalt,17.42340,1.244529,351
1,Breccia,10.11850,1.264813,959
2,Crustal,4.74469,0.103145,46


In [38]:
needed_samples_overview['Percentage (%)'] = needed_samples_overview['Number of Samples'] / needed_samples_overview['Number of Samples'].sum()
needed_samples_overview

,Type,Total weight (kg),Average weight (kg),Number of Samples,Percentage (%)
0,Basalt,17.42340,1.244529,351,0.258850
1,Breccia,10.11850,1.264813,959,0.707227
2,Crustal,4.74469,0.103145,46,0.033923


And finally, to tie it all back into a recommendation to the Artemis program, we can determine the average weight of samples we estimated in the preceding unit.

In [39]:
artemis_mission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Mission                          3 non-null      object 
 1   Total weight (kg)                3 non-null      int64  
 2   Payload (kg)                     3 non-null      int64  
 3   Sample weight from total (kg)    3 non-null      float64
 4   Sample weight from payload (kg)  3 non-null      float64
 5   Estimated sample weight (kg)     3 non-null      float64
dtypes: float64(3), int64(2), object(1)
memory usage: 272.0+ bytes


In [40]:
artemis_ave_weight = artemis_mission['Estimated sample weight (kg)'].mean()
artemis_ave_weight

63.61713411579792

In [41]:
needed_samples_overview['Weight to collect (kg)'] = artemis_ave_weight * needed_samples_overview['Percentage (%)']
needed_samples_overview

,Type,Total weight (kg),Average weight (kg),Number of Samples,Percentage (%),Weight to collect (kg)
0,Basalt,17.42340,1.244529,351,0.258850,16.467267
1,Breccia,10.11850,1.264813,959,0.707227,44.991764
2,Crustal,4.74469,0.103145,46,0.033923,2.158103


In [42]:
needed_samples_overview['Number of rocks to collect'] = needed_samples_overview['Weight to collect (kg)'] / needed_samples_overview['Average weight (kg)']
needed_samples_overview

,Type,Total weight (kg),Average weight (kg),Number of Samples,Percentage (%),Weight to collect (kg),Number of rocks to collect
0,Basalt,17.42340,1.244529,351,0.258850,16.467267,13.231731
1,Breccia,10.11850,1.264813,959,0.707227,44.991764,35.571884
2,Crustal,4.74469,0.103145,46,0.033923,2.158103,20.922917


So, we might tell the Artemis astronauts to please try to collect 13 Basalt rocks, 35 Breccia rocks, and 21 Crustal rocks. Whew!